In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# common imports
from __future__ import print_function
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
import sklearn
import tensorflow as tf
import torch
from matplotlib import gridspec
from scipy.stats import kde
from sklearn import decomposition
from torch import nn, optim
#from automlp import automlp
from automlp import automlp

import ipdb

In [3]:
from automlp import automlp, parameters

In [4]:
def make_model():
    return nn.Sequential(nn.Linear(28*28, 100),
                         nn.ReLU(),
                         nn.Linear(100, 10))

In [5]:
from torchvision.datasets import mnist
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class MnistAdapter(object):
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, index):
        inputs, targets = self.ds[index]
        return torch.FloatTensor(np.array(inputs)/255.0).reshape(-1), int(targets)

data = MnistAdapter(mnist.MNIST("./_data", download=True))
test_data = MnistAdapter(mnist.MNIST("./_data", download=True, train=False))

In [6]:
ps = parameters.ParameterSet()
ps += parameters.LogParameter("lr", 1e-5, 1e1)
ps += parameters.QuantizedLogParameter("bs", 5, 500)
ex = parameters.Exploration(ps)

In [10]:
for trial in range(200):
    pv = ex.pick_farthest()
    model = make_model()
    model.cuda()
    trainer = automlp.Trainer(model, "crossentropy")
    for epoch in range(30):
        trainer.train_dataset(data, options=pv, ntrain=3000000)
        loss = trainer.evaluate_classification(test_data)
    print(trial, loss, pv)
    ex.add_result(pv, loss)
    model.cpu()
    with open("mnist-100-sgd-{}.pth", "wb") as stream:
        torch.save(model, stream)

0 0.0989 OrderedDict([('lr', 1.718630436028197e-05), ('bs', 38)])


KeyboardInterrupt: 

In [ ]:
xscale('log'); yscale('log')
s = array([np.array(r[1].values()) for r in ex.results])
scatter(s[:,0], s[:,1], c=[r[0] for r in ex.results])

In [ ]:
min(ex.results)

In [ ]:
reload(parameters)
reload(automlp)
ps = parameters.ParameterSet()
ps += parameters.LogParameter("lr", 1e-5, 1e1)
ps += parameters.LogParameter("beta1", 1e-3, 3e-1)
ps += parameters.LogParameter("beta2", 1e-4, 3e-2)
ps += parameters.QuantizedLogParameter("bs", 20, 200)
ps += parameters.Constant("eps", 1e-8)
ex = parameters.Exploration(ps)

In [ ]:
for trial in range(200):
    pv = ex.pick_farthest()
    model = make_model()
    trainer = automlp.Trainer(model, "crossentropy")
    model.gpu()
    for epoch in range(30):
        trainer.train_dataset(data, options=options, ntrain=3000000)
        loss = trainer.evaluate_classification(test_data)
        print(i, loss, pv)
    ex.add_result(pv, loss)
    model.cpu()
    with open("mnist-100-adam-{}.pth", "wb") as stream:
        torch.save(model, stream)

In [ ]:
figsize(12,6)
s = array([np.array(r[1].values()) for r in ex.results])
subplot(121); xscale('log'); yscale('log'); scatter(s[:,0], s[:,1], c=[clip(r[0], 0, 0.03) for r in ex.results])
subplot(122); xscale('log'); yscale('log'); scatter(s[:,1], s[:,2], c=[clip(r[0], 0, 0.03) for r in ex.results])

In [ ]:
min(ex.results)